# Machine Learning Seminar
### Chapter07. Ensemble Learning and Random Forests
<br>
19.09.05  


JaeEun Yoo


<hr>

## Boosting

* Boosting (originally called hypothesis boosting) refers to any Ensemble method that can **combine several weak learners into a strong learner.** 
* The general idea of most boosting methods is to train predictors sequentially, each trying to correct its predecessor. 
* There are many boosting methods available, but by far the most popular are **AdaBoost** (short for Adaptive Boosting) and **Gradient Boosting**. Let’s start with AdaBoost.

### AdaBoost (Adaptive Boost)

* One way for **a new predictor to correct its predecessor** is to pay a bit more attention to the training instances that the predecessor underfitted. 
* This results in new predictors focusing more and more on the hard cases. 
* This is the technique used by AdaBoost.

![Figure 6-2](./img/07_01.PNG)

* For example, to build an AdaBoost classifier, a first base classifier (such as a Decision Tree) is trained and used to make predictions on the training set. 
* The relative weight of misclassified training instances is then increased. 
* A second classifier is trained using the updated weights and again it makes predictions on the training set, weights are updated, and so on.

![Figure 6-2](./img/07_02.PNG)

* Figure 7-8 shows the decision boundaries of five consecutive predictors on the moons dataset (in this example, each predictor is a highly regularized SVM classifier with an RBF kernel14). 
* The first classifier gets many instances wrong, so their **weights get boosted.** 
* The second classifier therefore does a better job on these instances, and so on. 
* The plot on the right represents the same sequence of predictors except that the learning rate is halved 
    * (i.e., the misclassified instance weights are boosted half as much at every iteration). 
* As you can see, this sequential learning technique has some similarities with **Gradient Descent**, except that instead of tweaking a single predictor’s parameters to minimize a cost function, AdaBoost **adds predictors to the ensemble**, gradually making it better.
* Once all predictors are trained, the ensemble makes predictions very much like bagging or pasting, except that **predictors have different weights depending on their overall accuracy on the weighted training set.**

> WARNING

There is one important drawback to this sequential learning technique: **it cannot be parallelized (or only partially),** since each predictor can only be trained after the previous predictor has been trained and evaluated.  
As a result, **it does not scale as well as bagging or pasting.**

* Let’s take a closer look at the **AdaBoost algorithm.**  
* **Each instance weight w<sup>(i)</sup>** is initially set to 1/m.  
* A first predictor is trained and its **weighted error rate r<sub>1</sub>** is computed on the training set; see Equation 7-1.

![Figure 6-2](./img/07_03.PNG)

* **The predictor’s weight α<sub>j</sub>** is then computed using Equation 7-2, where **η is the learning rate hyperparameter** (defaults to 1).

![Figure 6-2](./img/07_04.PNG)

* The more accurate the predictor is, the higher its weight will be. 
* If it is just guessing randomly, then its weight will be close to zero.
* However, if it is most often wrong (i.e., less accurate than random guessing), then its weight will be negative.

![Figure 6-2](./img/07_05.PNG)

* The instance weights are updated using Equation 7-3: the misclassified instances are boosted.
* Then all the instance weights are normalized.


(i.e., divided by ![Figure 6-2](./img/07_06.PNG)

* Finally, a new predictor is trained using the updated weights, and the whole process is repeated (the new predictor’s weight is computed, the instance weights are updated, then another predictor is trained, and so on). 
* The algorithm stops **when the desired number of predictors is reached, or when a perfect predictor is found.**
* To make predictions, AdaBoost simply computes the predictions of all the predictors and weighs them using the predictor weights α<sub>j</sub>. 
* The predicted class is the one that **receives the majority of weighted votes** (see Equation 7-4).

![Figure 6-2](./img/07_07.PNG)

* Scikit-Learn actually uses a multiclass version of AdaBoost called *SAMME* (which stands for *Stagewise Additive Modeling using a Multiclass Exponential loss function*). 
    * When there are just two classes, SAMME is equivalent to AdaBoost.
* Moreover, if the predictors can estimate class probabilities (i.e., if they have a **predict_proba()** method), Scikit-Learn can use a variant of SAMME called *SAMME.R* (the R stands for “Real”), which relies on class probabilities rather than predictions and generally performs better.
* The following code trains an AdaBoost classifier based on 200 *Decision Stumps* using Scikit-Learn’s AdaBoostClassifier class (as you might expect, there is also an AdaBoostRegressor class). 
* A Decision Stump is a Decision Tree with max_depth=1
    * In other words, a tree composed of a single decision node plus two leaf nodes. 
* This is the default base estimator for the AdaBoostClassifier class:

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons

In [2]:
X,y = make_moons()

In [3]:
make_moons

<function sklearn.datasets.samples_generator.make_moons(n_samples=100, shuffle=True, noise=None, random_state=None)>

In [39]:
moons

(array([[ 4.81607432e-01, -3.55142763e-01],
        [-6.72300890e-01,  7.40277997e-01],
        [-5.18392568e-01,  8.55142763e-01],
        [ 1.34536505e+00, -4.38468422e-01],
        [ 1.03205158e+00, -4.99486216e-01],
        [-3.45365054e-01,  9.38468422e-01],
        [-9.97945393e-01,  6.40702200e-02],
        [ 9.97945393e-01,  6.40702200e-02],
        [ 0.00000000e+00,  5.00000000e-01],
        [ 9.67948422e-01, -4.99486216e-01],
        [ 9.03976974e-01, -4.95379113e-01],
        [ 9.90311321e-02,  6.61162609e-02],
        [ 7.77479066e-01, -4.74927912e-01],
        [ 1.62348980e+00, -2.81831482e-01],
        [ 8.71318704e-01,  4.90717552e-01],
        [ 1.09602303e+00, -4.95379113e-01],
        [ 6.54634946e-01, -4.38468422e-01],
        [-9.60230259e-02,  9.95379113e-01],
        [ 3.45365054e-01,  9.38468422e-01],
        [-4.62538290e-01,  8.86599306e-01],
        [-9.67294863e-01,  2.53654584e-01],
        [ 1.57211666e+00, -3.20172255e-01],
        [ 7.15472413e-01, -4.586

In [44]:
ada_clf = AdaBoostClassifier(
            DecisionTreeClassifier(max_depth=1),
            n_estimators=200,
            algorithm="SAMME.R",
            learning_rate=0.5)

In [45]:
ada_clf.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [89]:
ada_clf.predict([[0.5,0.5]])

array([0], dtype=int64)

In [48]:
ada_clf.score(X,y)

1.0

> TIP

If your AdaBoost ensemble is overfitting the training set, you can try **reducing the number of estimators** or **more strongly regularizing the base estimator.**

<br>

### Gradient Boosting

* Another very popular Boosting algorithm is **Gradient Boosting**. 
* Just like AdaBoost, Gradient Boosting works by sequentially adding predictors to an ensemble, each one correcting its predecessor. 
* However, instead of tweaking the instance weights at every iteration like AdaBoost does, **this method tries to fit the new predictor to the residual errors made by the previous predictor.**
* Let’s go through a simple regression example using Decision Trees as the base predictors (of course Gradient Boosting also works great with regression tasks). 
* This is called *Gradient Tree Boosting*, or *Gradient Boosted Regression Trees (GBRT)*. 
* First, let’s fit a DecisionTreeRegressor to the training set (for example, a noisy quadratic training set):

In [18]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [19]:
X.shape

(100, 2)

In [20]:
y.shape

(100,)

Now train a second DecisionTreeRegressor on the residual errors made by the first predictor:

In [5]:
y2 = y - tree_reg1.predict(X)

tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Then we train a third regressor on the residual errors made by the second predictor:

In [6]:
y3 = y2 - tree_reg2.predict(X)

tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Now we have an ensemble containing three trees.  
It can make predictions on a new instance simply by adding up the predictions of all the trees:

In [23]:
y_pred = sum(tree.predict([[2,0.99]]) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [24]:
y_pred

array([0.88461538])

![Figure 6-2](./img/07_08.PNG)

* Figure 7-9 represents the predictions of these three trees in the left column, and the ensemble’s predictions in the right column. 
    * In the first row, the ensemble has just one tree, so its predictions are exactly the same as the first tree’s predictions. 
    * In the second row, a new tree is trained on the residual errors of the first tree.
    * On the right you can see that the ensemble’s predictions are equal to the sum of the predictions of the first two trees. 
    * Similarly, in the third row another tree is trained on the residual errors of the second tree. 
    * You can see that the ensemble’s predictions gradually get better as trees are added to the ensemble.

<br>

* A simpler way to train GBRT ensembles is to use Scikit-Learn’s *GradientBoostingRegressor* class. 
* Much like the RandomForestRegressor class, it has hyperparameters to control the growth of Decision Trees (e.g., max_depth, min_samples_leaf, and so on), as well as hyperparameters to control the ensemble training, such as the number of trees (n_estimators). 
* The following code creates the same ensemble as the previous one:

In [25]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

* The *learning_rat*e hyperparameter scales the contribution of each tree. 
    * If you set it to a low value, such as 0.1, you will need more trees in the ensemble to fit the training set, but the predictions will usually generalize better. 
* This is a regularization technique called **shrinkage**. 


![Figure 6-2](./img/07_09.PNG)

* Figure 7-10 shows two GBRT ensembles trained with a low learning rate: 
    * the one on the left does not have enough trees to fit the training set,
    * while the one on the right has too many trees and overfits the training set.

* In order to find the optimal number of trees, you can use early stopping (see Chapter 4). 
* A simple way to implement this is to use the *staged_predict()* method:
    * it returns an iterator over the predictions made by the ensemble at each stage of training (with one tree, two trees, etc.). 
* The following code trains a GBRT ensemble with 120 trees, then measures the validation error at each stage of training to find the optimal number of trees, and finally trains another GBRT ensemble using the optimal number of trees:

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [27]:
X_train, X_val, y_train, y_val = train_test_split(X, y)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)


(75, 2)
(75,)
(25, 2)
(25,)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=120,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]

In [30]:
errors

[0.23238276440376182,
 0.20520185375519062,
 0.1817077849458973,
 0.16396975330990482,
 0.15033928039124006,
 0.13756787904158294,
 0.12723950776027335,
 0.1185297248976464,
 0.10971371888891925,
 0.10378305075899451,
 0.09915788652829102,
 0.09354451078104026,
 0.09046391374451458,
 0.09042533714502157,
 0.0882195461968501,
 0.08654023471338698,
 0.08377402515928267,
 0.08496718241017817,
 0.08699624951509458,
 0.08394975347957175,
 0.08286519086369207,
 0.07700649093529138,
 0.07449857468217595,
 0.0732159674370101,
 0.07247582942931458,
 0.07045167557711193,
 0.06573582789892654,
 0.06485453910284955,
 0.06102975192821259,
 0.05976172930727478,
 0.05412256756865735,
 0.052750250264340146,
 0.05211702047113812,
 0.04944743425350511,
 0.05071141608764854,
 0.04987457073331363,
 0.049571289249545375,
 0.04553191579163353,
 0.04463756020531218,
 0.04366662141926428,
 0.04186461499990637,
 0.04157574056400914,
 0.04104529791628604,
 0.03807905347087447,
 0.03739119154756834,
 0.034940593

In [16]:
len(errors)

120

![Figure 6-2](./img/07_11.PNG)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

In [80]:
print(X_val.shape)
cnt = 0
for y_pred in gbrt.staged_predict(X_val):
    print(y_pred)
    cnt+=1
print(cnt)

(25, 2)
[0.5649697  0.47610811 0.47610811 0.5649697  0.47610811 0.47610811
 0.47610811 0.5649697  0.47610811 0.47610811 0.47610811 0.47610811
 0.47610811 0.47610811 0.47610811 0.5649697  0.5649697  0.47610811
 0.568      0.568      0.47610811 0.468      0.5649697  0.5649697
 0.47610811]
[0.60544242 0.43660541 0.43660541 0.60544242 0.43660541 0.43660541
 0.43660541 0.60544242 0.43660541 0.43660541 0.43660541 0.43660541
 0.43660541 0.43660541 0.43660541 0.60544242 0.60544242 0.43660541
 0.6112     0.6112     0.43660541 0.4212     0.60544242 0.60544242
 0.43660541]
[0.64026462 0.39882722 0.39882722 0.64026462 0.39882722 0.39882722
 0.39882722 0.64026462 0.39882722 0.39882722 0.39882722 0.39882722
 0.39882722 0.39882722 0.39882722 0.64026462 0.64026462 0.39882722
 0.65008    0.65008    0.39882722 0.37830973 0.64026462 0.64026462
 0.39882722]
[0.67320785 0.36667399 0.36667399 0.67320785 0.36667399 0.36667399
 0.36667399 0.67320785 0.36667399 0.36667399 0.36667399 0.36667399
 0.36667399 0.36

In [76]:
y_pred.shape

(25,)

In [32]:
bst_n_estimators = np.argmin(errors)
print('best estimators number : ',bst_n_estimators+1)

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

best estimators number :  120


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=119,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

![Figure 6-2](./img/07_10.PNG)

> The validation errors are represented on the left of Figure 7-11, and the best model’s predictions are
represented on the right.

* It is also possible to implement early stopping by actually stopping training early (instead of training a large number of trees first and then looking back to find the optimal number). 
* You can do so by setting warm_start=True, which makes Scikit-Learn **keep existing trees when the fit() method is called**, allowing incremental training. 
* The following code stops training when the validation error does not improve for five iterations in a row:

In [38]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf") #infinite number
error_going_up = 0


In [39]:
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping

In [40]:
n_estimators

119

* The GradientBoostingRegressor class also supports a *subsample* hyperparameter, which **specifies the fraction of training instances to be used for training each tree**. 
* For example, if subsample=0.25, then each tree is trained on 25% of the training instances, selected randomly. 
* As you can probably guess by now, this trades **a higher bias for a lower variance.** 
* It also speeds up training considerably. 
* This technique is called **Stochastic Gradient Boosting.**

>NOTE

It is possible to use Gradient Boosting with other cost functions. This is controlled by the loss hyperparameter (see Scikit-
Learn’s documentation for more details).

<hr>

## Stacking

* The last Ensemble method we will discuss in this chapter is called **stacking** (short for stacked generalization).
* It is based on a simple idea: 
    * instead of using trivial functions (such as hard voting) to aggregate the predictions of all predictors in an ensemble, **why don’t we train a model to perform this aggregation?** 


![Figure 6-2](./img/07_12.PNG)

* Figure 7-12 shows such an ensemble performing a regression task on a new instance. 
    * Each of the bottom three predictors predicts a different value (3.1, 2.7, and 2.9), and **then the final predictor (called a *blender*, or a meta learner) takes these predictions as inputs and makes the final prediction (3.0).**

* To train the blender, a common approach is to use a **hold-out set.** Let’s see how it works. 

![Figure 6-2](./img/07_13.PNG)

* First, the training set is split in two subsets. The first subset is used to train the predictors in the first layer (see Figure 7-13).

![Figure 6-2](./img/07_14.PNG)

* Next, the first layer predictors are used to make predictions on the second (held-out) set (see Figure 7-14). 
    * This ensures that **the predictions are “clean,” since the predictors never saw these instances during training.** 
* Now for each instance in the hold-out set there are three predicted values.
* We can create **a new training set using these predicted values as input features** (which makes this new training set **three-dimensional**), and keeping the target values. 
* The blender is trained on this new training set, so it learns to predict the target value given the first layer’s predictions.

* It is actually possible to train several different blenders this way (e.g., one using Linear Regression, another using Random Forest Regression, and so on): we get a whole layer of blenders. 
* The trick is to split the training set into three subsets: 
    * the first one is used to train the first layer
    * the second one is used to create the training set used to train the second layer (using predictions made by the predictors of the first layer)
    * the third one is used to create the training set to train the third layer(using predictions made by the predictors of the second layer). 
* Once this is done, we can make a prediction for a new instance by going through each layer sequentially, as shown in Figure 7-15.

![Figure 6-2](./img/07_15.PNG)

* Unfortunately, **Scikit-Learn does not support stacking directly**, but it is not too hard to roll out your own implementation .
* Alternatively, you can use an open source implementation such as brew (available at https://github.com/viisar/brew).